In [2]:
import json
import pandas as pd
import numpy as np
from urllib import parse
import requests
from bs4 import BeautifulSoup as bs
import re

In [4]:
# 여행 카테고리 top10
res = requests.get('https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&sca=%EA%B5%AD%EB%82%B4%2F%ED%95%B4%EC%99%B8%2F%EC%97%AC%ED%96%89')
soup = bs(res.text, 'html.parser')

channel_list = soup.select('table .subject h1 a')[0:10]

channel_list = [x.text.strip() for x in channel_list]

In [5]:
channel_list

['빠니보틀 Pani Bottle',
 '곽튜브',
 'Rirang OnAir',
 '코이티비 KOITV',
 'EBS 컬렉션 - 라이프스타일',
 'Sophie Kim _하은',
 '진정부부',
 '끼룩푸드 seagull food',
 '원지의하루',
 '브레이너 제이의 숙면 여행']

In [6]:
count_list = soup.select('table .subject h3')[0:10]
subs = []
view = []
video = []
search = []

for i in count_list :
    a, b, c = i.text.split()
    s_v_split = re.split(r'^(\d{1,3}[만])', a)
    subs.append(int(s_v_split[1].split('만')[0]))
    view.append(int(''.join(s_v_split[2][:-1].split('억'))))
    video.append(int(b.replace(',', '')))
    search.append(int(c.replace(',', '')))

print(subs)
print(view)
print(video)
print(search)

[190, 167, 131, 115, 94, 89, 86, 82, 78, 68]
[42516, 37643, 21006, 99444, 63142, 2969, 36335, 57380, 14445, 13431]
[301, 311, 269, 1878, 3698, 129, 352, 695, 351, 200]
[4894, 1798, 5344, 10874, 5578, 4930, 230, 647, 3142, 3916]


In [9]:
youtuber_rank = pd.DataFrame()

youtuber_rank['channel_name'] = channel_list
youtuber_rank['subscriber (10th)'] = subs
youtuber_rank['cum_view (10th)'] = view
youtuber_rank['video'] = video
youtuber_rank['search'] = search
youtuber_rank['category'] = '국내/해외/여행'

In [11]:
youtuber_rank.to_csv('./data/youtuber_rank_top10.csv', encoding='utf-8-sig', index=False)

In [45]:
# 사용된 유튜버 채널
res = requests.get('https://youtube-rank.com/board/bbs/search.php?sfl=wr_subject%7C%7Cwr_content%7C%7Cdescription&stx=%EB%B0%B1%EC%A2%85%EC%9B%90')
soup = bs(res.text, 'html.parser')

channel_list = soup.select('table .subject h1 a')[0]
channel_name = channel_list.text.strip().split('\n')[1]

count = soup.select('table .subject h3')[0]
subs = []
view = []
video = []
search = []

subs, view, video = count.text.split()

subs = int(subs.split('만')[0])
view = int(''.join(view[:-1].split('억')))
video = int(video.replace(',', ''))

baek = [channel_name, subs, view, video, 0, '음식/요리/레시피']

In [79]:
res = requests.get('https://youtube-rank.com/board/bbs/search.php?sfl=wr_subject%7C%7Cwr_content%7C%7Cdescription&stx=%ED%86%A0%EB%AA%A8%ED%86%A0%EB%AA%A8')
soup = bs(res.text, 'html.parser')

channel_list = soup.select('table .subject h1 a')[0]
channel_name = channel_list.text.strip().split('\n')[1]

count = soup.select('table .subject h3')[0]

subs, view, video = count.text.split()

subs = int(subs.split('만')[0])
view = int(''.join(view[:-1].split('억')))
video = int(video.replace(',', ''))
category = soup.select('table .subject h1 a span')[0].text

category = re.sub('\[|\]', '', category)

tomotomo = [channel_name, subs, view, video, 0, category]

In [77]:
res = requests.get('https://youtube-rank.com/board/bbs/search.php?srows=10&gr_id=&sfl=wr_subject%7C%7Cwr_content%7C%7Cdescription%7C%7Cwr_link1&stx=%EC%97%AC%ED%96%89%EA%B0%80+%EC%A0%9C%EC%9D%B4&sop=and')
soup = bs(res.text, 'html.parser')

channel_list = soup.select('table .subject h1 a')[0]
channel_name = channel_list.text.strip().split('\n')[1]

count = soup.select('table .subject h3')[0]

subs, view, video = count.text.split()

subs = int(subs.split('만')[0])
view = int(''.join(view[:-1].split('억')))
video = int(video.replace(',', ''))
category = soup.select('table .subject h1 a span')[0].text

category = re.sub('\[|\]', '', category)

jay = [channel_name, subs, view, video, 0, category]

In [80]:
tomotomo

['토모토모TomoTomo', 103, 23139, 206, 0, '미분류']

In [81]:
additional = pd.DataFrame([baek, tomotomo, jay], 
             columns = ['channel_name', 'subscriber (10th)', 'cum_view (10th)', 'video', 'search', 'category'])

used_youtuber = youtuber_rank.append(additional, ignore_index=True)

C:\Users\TT21\AppData\Local\Temp\ipykernel_48168\2991888735.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  used_youtuber = youtuber_rank.append(additional, ignore_index=True)


In [82]:
used_youtuber

,channel_name,subscriber (10th),cum_view (10th),video,search,category
0,빠니보틀 Pani Bottle,190,42516,301,4894,국내/해외/여행
1,곽튜브,167,37643,311,1798,국내/해외/여행
2,Rirang OnAir,131,21006,269,5344,국내/해외/여행
3,코이티비 KOITV,115,99444,1878,10874,국내/해외/여행
4,EBS 컬렉션 - 라이프스타일,94,63142,3698,5578,국내/해외/여행
5,Sophie Kim _하은,89,2969,129,4930,국내/해외/여행
6,진정부부,86,36335,352,230,국내/해외/여행
7,끼룩푸드 seagull food,82,57380,695,647,국내/해외/여행
8,원지의하루,78,14445,351,3142,국내/해외/여행
9,브레이너 제이의 숙면 여행,68,13431,200,3916,국내/해외/여행


In [83]:
used_youtuber = used_youtuber.drop([2, 4, 5, 6, 7, 8, 9])

In [84]:
used_youtuber = used_youtuber.sort_values('subscriber (10th)', ascending=False)

In [85]:
used_youtuber

,channel_name,subscriber (10th),cum_view (10th),video,search,category
10,백종원 PAIK JONG WON,592,98491,559,0,음식/요리/레시피
0,빠니보틀 Pani Bottle,190,42516,301,4894,국내/해외/여행
1,곽튜브,167,37643,311,1798,국내/해외/여행
3,코이티비 KOITV,115,99444,1878,10874,국내/해외/여행
11,토모토모TomoTomo,103,23139,206,0,미분류
12,여행가 제이 [Jay world traveler],60,14456,379,0,국내/해외/여행


In [86]:
used_youtuber.to_csv('./data/used_youtuber.csv', encoding='utf-8-sig', index=False)